In [20]:
import collections
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [2]:
data = np.load('../deep_learning_trainer/trained_model/1s_win_length_1imgStep/3_images/training/training_small_nImg_3_nSteady_50_1sWin_1imgStep.npz')

In [5]:
input_shape=(3, 30, 30, 1)
kernel_size = (3, 3)
n_outputs = 2
model = models.Sequential()
model.add(layers.ConvLSTM2D(filters=64, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(n_outputs, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 28, 28, 64)        150016    
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               5017700   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 5,167,918
Trainable params: 5,167,918
Non-trainable params: 0
_________________________________________________________________


In [14]:
X = data['X']
y = data['y']
meta = data['meta']
y = tf.keras.utils.to_categorical(y)

In [16]:
ix = np.random.choice(range(len(y)), size=len(y), replace=False)

In [17]:
X = X[:, :, :, :, np.newaxis][ix]
y = y[ix]
meta = meta[ix]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
y_integers = np.argmax(y_train, axis=1)
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_integers), y_integers)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train[:50000], y_train[:50000], epochs=10, 
                    validation_split=0.2, batch_size=256, class_weight=class_weight_dict)